In [2]:
import requests
from glob import glob
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from time import sleep
# import re

In [3]:
# http://www.networkinghowtos.com/howto/common-user-agent-list/
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)', #'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

In [4]:
url = "https://www.amazon.com/ANYOYO-Outdoor-Toboggan-Downhill-Screamer/dp/B07XG4HJSR/ref=sr_1_53?dchild=1&keywords=snow+sled&qid=1595968264&sr=8-53" #'https://www.amazon.es/Nikon-D750-AF-S-24-120-VR/dp/B00NMD804A/ref=sr_1_4?__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=nikon+24+120&qid=1590338558&quartzVehicle=106-1339&replacementKeywords=nikon+24&sr=8-4'

In [5]:
page = requests.get(url, headers=HEADERS)
soup = BeautifulSoup(page.content, features="lxml")

In [6]:
soup

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-us"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link crossorigin="" href="https://images-na.ssl-images-amazon.com" rel="preconnect"/>
<link crossorigin="" href="https://m.media-amazon.com" rel="preconnect"/>
<link crossorigin="" href="https://completion.amazon.com" rel="preconnect"/>
<!-- sp:feature:aui-assets -->
<link href="https://images-na.ssl-images-amazon.com/images/I/11EIQ5IGqaL._RC|01ZTHTZObnL.css,017DsKjNQJL.css,01NL3TVKoYL.css,01HEsUOLYvL.css,41EWOOlBJ9L.css,11rkuLsEFQL.css,01ElnPiDxWL.css,11QxHU4QYaL.css,01Sp8sB1HiL.css,01IdKcBuAdL.css,01y-XAlI+2L.css,01evdoiemkL.css,01K+Ps1DeEL.css,314djKvMsUL.css,01W6EiNzKkL.css,51AZ-Jz5kmL.css,11UGC+GXOPL.css,21LK7jaicML.css,11L58Qpo0GL.css,21kyTi1FabL.css,01ruG+gDPFL.css,01YhS3Cs-hL.css,21GwE3cR-yL.css,0

In [13]:
prod_tracker = pd.read_csv('trackers/TRACKER_PRODUCTS.csv', sep=';')
prod_tracker_URLS = prod_tracker.url
display(prod_tracker.shape)
prod_tracker

(80, 3)

,url,code,buy_below
0,https://www.amazon.com/ANYOYO-Outdoor-Toboggan...,snow_sled,10.0
1,https://www.amazon.com/AIRHEAD-CLASSIC-Plastic...,airhead_classic_snow_sled,10.0
2,https://www.amazon.com/Slippery-Racer-Downhill...,slippery_racer_snow_sled,30.0
3,https://www.amazon.com/Flexible-Flyer-Lightwei...,flyer_611_snow_sled,5.0
4,https://www.amazon.com/DUDU-Snow-Tube-Inflatab...,adudu_47_snow_tube,20.0
...,...,...,...
75,https://www.amazon.com/Columbia-Watertight-Jac...,columbia_watertight_rain_jacket,20.0
76,https://www.amazon.com/Baleaf-Packable-Waterpr...,baleaf_rain_jacket,10.0
77,https://www.amazon.com/Frogg-Toggs-Ultra-Lite2...,frogg_toggs_rain_jacket,10.0
78,https://www.amazon.com/SWISSWELL-Waterproof-Wi...,swisswell_rain_jacket,10.0


In [9]:
tracker_log = pd.DataFrame()

###### tracking function - keeps cycling through url's

for x, url in enumerate(prod_tracker_URLS):
    page = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(page.content, features="lxml")
    title = soup.find(id='productTitle').get_text().strip()
    try:
        price = float(soup.find(id='priceblock_ourprice').get_text().replace('.', '').replace('€', '').replace(',', '.').strip())
    except:
        price = ''
    review_score = float(soup.select('.a-star-4')[0].get_text().split(' ')[0].replace(",", "."))
    review_count = int(soup.select('#acrCustomerReviewText')[0].get_text().split(' ')[0].replace(".", "").replace(",", ""))
    try:
        soup.select('#availability .a-color-state')[0].get_text().strip()
        stock = 'Out of Stock'
    except:
        stock = 'Available'
    
    now = datetime.now().strftime('%Y-%m-%d %Hh%Mm')
    log = pd.DataFrame({'date': now,
                        'code': prod_tracker.code[x],
                        'url': url,
                        'title': title,
                        'price': price,
                        'alert': prod_tracker.buy_below[x],
                        'review_score': review_score,
                        'review_count': review_count}, index=[x])

    tracker_log = tracker_log.append(log)
    print('appended '+ prod_tracker.code[x] +'\n' + title + '\n\n')
    sleep(5)

appended snow_sled
ANYOYO Kids Outdoor Plastic Sport Toboggan Winter Snow Sled Board w/Pull Rope,Pull Ropes for Outdoor Winter Slider Downhill Snow Board,Baby Pull Sled Sled Slider Snow Screamer, Blue 35" x 18" x 5"


appended airhead_classic_snow_sled
Airhead Classic | 1-2 Person Snow Sled


appended slippery_racer_snow_sled
Slippery Racer Downhill Xtreme Toboggan Snow Sled


appended flyer_611_snow_sled
Flexible Flyer 611 Flying Carpet Lightweight Roll Up Snow Sled Portable Rolling Snow Slider, Blue


appended adudu_47_snow_tube
A-DUDU Snow Tube - Super Big 47 Inch Inflatable Snow Sled for Kids and Adults - Heavy Duty Inflatable Snow Tube Made by Thickening Material of 0.6mm


appended emsco_group_snowboard
Emsco Group ESP 110 cm Freeride Snowboard - Adjustable Bindings - for Beginners and Experienced Riders, Graphic


appended voltsurf_snowboard
VoltSurf All-Mountain Snowboard (SINTERED Base)


appended emsco_group_kids_snowboard
EMSCO Group – Graffiti Snowboard – Great for Beginner

In [17]:
tracker_log.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          1 non-null      object 
 1   code          1 non-null      object 
 2   url           1 non-null      object 
 3   title         1 non-null      object 
 4   price         1 non-null      object 
 5   alert         1 non-null      int64  
 6   review_score  1 non-null      float64
 7   review_count  1 non-null      int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 72.0+ bytes


In [18]:
tracker_log.to_excel('search_history/SEARCH_HISTORY.xlsx', index=False)

In [14]:
search_product_list()

appended snow_sled
ANYOYO Kids Outdoor Plastic Sport Toboggan Winter Snow Sled Board w/Pull Rope,Pull Ropes for Outdoor Winter Slider Downhill Snow Board,Baby Pull Sled Sled Slider Snow Screamer, Blue 35" x 18" x 5"


appended airhead_classic_snow_sled
Airhead Classic | 1-2 Person Snow Sled


appended slippery_racer_snow_sled
Slippery Racer Downhill Xtreme Toboggan Snow Sled


appended flyer_611_snow_sled
Flexible Flyer 611 Flying Carpet Lightweight Roll Up Snow Sled Portable Rolling Snow Slider, Blue


appended adudu_47_snow_tube
A-DUDU Snow Tube - Super Big 47 Inch Inflatable Snow Sled for Kids and Adults - Heavy Duty Inflatable Snow Tube Made by Thickening Material of 0.6mm


appended emsco_group_snowboard
Emsco Group ESP 110 cm Freeride Snowboard - Adjustable Bindings - for Beginners and Experienced Riders, Graphic


appended voltsurf_snowboard
VoltSurf All-Mountain Snowboard (SINTERED Base)


appended emsco_group_kids_snowboard
EMSCO Group – Graffiti Snowboard – Great for Beginner

appended sperry_rain_boots
Sperry Women's Saltwater Crunch Rain Boot


appended tingley_rain_boots
Tingley 31151 Economy SZ7 Kneed Boot for Agriculture, 15-Inch, Black


appended servus_rain_boots
Servus Comfort Technology 14" PVC Steel Toe Men's Work Boots


appended clc_rain_boots
CLC Custom Leathercraft Rain Wear R23008 Over The Sock Black PVC Men's Rain Boot, Size 8


appended tidewe_rain_boots
TideWe Rubber Boots for Men Multi-Season, Waterproof Muck Rain Boots with Steel Shank, 6mm Neoprene Durable Rubber Neoprene Outdoor Hunting Boots Realtree Edge Camo (Black, Brown & Camo)


appended honeywell_rain_boots
Servus XTP 15" PVC Chemical-Resistant Steel Toe Men's Work Boots, Black, Yellow & Gray, 11


appended quicksilk_rain_boots
Quicksilk EYUSHIJIA Men's Waterproof Snow Boots Hiking Boot


appended columbia_arcadia_rain_jacket
Columbia Women's Arcadia Ii Waterproof Breathable Jacket With Packable Hood


appended columbia_switchback_rain_jacket
Columbia Women's Switchback Lined Lon

In [27]:
start_urls = ["https://www.amazon.com/s?k=snow+sled&page=" + str(i) + "&qid=1595968264&ref=sr_pg_2" for i in range(2, 101)]
start_urls
# "https://www.amazon.com/s?k=snow+sled&ref=nb_sb_noss_1"

['https://www.amazon.com/s?k=snow+sled&page=2&qid=1595968264&ref=sr_pg_2',
 'https://www.amazon.com/s?k=snow+sled&page=3&qid=1595968264&ref=sr_pg_2',
 'https://www.amazon.com/s?k=snow+sled&page=4&qid=1595968264&ref=sr_pg_2',
 'https://www.amazon.com/s?k=snow+sled&page=5&qid=1595968264&ref=sr_pg_2',
 'https://www.amazon.com/s?k=snow+sled&page=6&qid=1595968264&ref=sr_pg_2',
 'https://www.amazon.com/s?k=snow+sled&page=7&qid=1595968264&ref=sr_pg_2',
 'https://www.amazon.com/s?k=snow+sled&page=8&qid=1595968264&ref=sr_pg_2',
 'https://www.amazon.com/s?k=snow+sled&page=9&qid=1595968264&ref=sr_pg_2',
 'https://www.amazon.com/s?k=snow+sled&page=10&qid=1595968264&ref=sr_pg_2',
 'https://www.amazon.com/s?k=snow+sled&page=11&qid=1595968264&ref=sr_pg_2',
 'https://www.amazon.com/s?k=snow+sled&page=12&qid=1595968264&ref=sr_pg_2',
 'https://www.amazon.com/s?k=snow+sled&page=13&qid=1595968264&ref=sr_pg_2',
 'https://www.amazon.com/s?k=snow+sled&page=14&qid=1595968264&ref=sr_pg_2',
 'https://www.amazon

# Code from gists in article

In [9]:
import requests
from glob import glob
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from time import sleep

# http://www.networkinghowtos.com/howto/common-user-agent-list/
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

# imports a csv file with the url's to scrape
prod_tracker = pd.read_csv('trackers/TRACKER_PRODUCTS.csv', sep=';')
prod_tracker_URLS = prod_tracker.url

# fetch the url
page = requests.get(prod_tracker_URLS[0], headers=HEADERS)

# create the object that will contain all the info in the url
soup = BeautifulSoup(page.content, features="lxml")

In [10]:
# product title
title = soup.find(id='productTitle').get_text().strip()

# to prevent script from crashing when there isn't a price for the product
try:
    price = float(soup.find(id='priceblock_ourprice').get_text().replace('.', '').replace('€', '').replace(',', '.').strip())
except:
    price = ''

# review score
review_score = float(soup.select('.a-star-4-5')[0].get_text().split(' ')[0].replace(",", "."))

# how many reviews
review_count = int(soup.select('#acrCustomerReviewText')[0].get_text().split(' ')[0].replace(".", ""))

# checking if there is "Out of stock" and if not, it means the product is available
try:
    soup.select('#availability .a-color-state')[0].get_text().strip()
    stock = 'Out of Stock'
except:
    stock = 'Available'

In [11]:
# product title
title = soup.find(id='productTitle').get_text().strip()
title

'Nikon D750 + AF-S 24-120 VR - Cámara Digital'

In [12]:
# to prevent script from crashing when there isn't a price for the product
try:
    price = float(soup.find(id='priceblock_ourprice').get_text().replace('.', '').replace('€', '').replace(',', '.').strip())
except:
    price = ''
price

1727.0

In [13]:
# review score
review_score = float(soup.select('.a-star-4-5')[0].get_text().split(' ')[0].replace(",", "."))

# how many reviews
review_count = int(soup.select('#acrCustomerReviewText')[0].get_text().split(' ')[0].replace(".", ""))

print(str(review_score) + ' out of ' + str(review_count) + ' reviews')

4.4 out of 64 reviews


In [14]:
# checking if there is "Out of stock" and if not, it means the product is available
try:
    soup.select('#availability .a-color-state')[0].get_text().strip()
    stock = 'Out of Stock'
except:
    stock = 'Available'
stock

'Available'

# Amazon_Scraper.py code

You can copy the different sections to separate cells to understand what the variables are storing. Useful also to quickly eliminate bugs that naturally appear over time!

In [10]:
import requests
from glob import glob
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from time import sleep

# http://www.networkinghowtos.com/howto/common-user-agent-list/
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)', #'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})


def search_product_list(interval_count = 1, interval_hours = 6):
    """
    This function lods a csv file named TRACKER_PRODUCTS.csv, with headers: [url, code, buy_below]
    It looks for the file under in ./trackers
    
    It also requires a file called SEARCH_HISTORY.xslx under the folder ./search_history to start saving the results.
    An empty file can be used on the first time using the script.
    
    Both the old and the new results are then saved in a new file named SEARCH_HISTORY_{datetime}.xlsx
    This is the file the script will use to get the history next time it runs.

    Parameters
    ----------
    interval_count : TYPE, optional
        DESCRIPTION. The default is 1. The number of iterations you want the script to run a search on the full list.
    interval_hours : TYPE, optional
        DESCRIPTION. The default is 6.

    Returns
    -------
    New .xlsx file with previous search history and results from current search

    """
    prod_tracker = pd.read_csv('trackers/TRACKER_PRODUCTS.csv', sep=';')
    prod_tracker_URLS = prod_tracker.url
    tracker_log = pd.DataFrame()
    now = datetime.now().strftime('%Y-%m-%d %Hh%Mm')
    interval = 0 # counter reset
    
    while interval < interval_count:

        for x, url in enumerate(prod_tracker_URLS):
            page = requests.get(url, headers=HEADERS)
            soup = BeautifulSoup(page.content, features="lxml")
            
            #product title
            title = soup.find(id='productTitle').get_text().strip()
            
            # to prevent script from crashing when there isn't a price for the product
            try:
                price = float(soup.find(id='priceblock_ourprice').get_text().replace('.', '').replace('€', '').replace(',', '.').strip())
            except:
                # this part gets the price in dollars from amazon.com store
                try:
                    price = float(soup.find(id='priceblock_saleprice').get_text().replace('$', '').replace(',', '').strip())
                except:
                    price = ''

            try:
                review_score = float(soup.select('i[class*="a-icon a-icon-star a-star-"]')[0].get_text().split(' ')[0].replace(",", "."))
                review_count = int(soup.select('#acrCustomerReviewText')[0].get_text().split(' ')[0].replace(".", ""))
            except:
                # sometimes review_score is in a different position... had to add this alternative with another try statement
                try:
                    review_score = float(soup.select('i[class*="a-icon a-icon-star a-star-"]')[1].get_text().split(' ')[0].replace(",", "."))
                    review_count = int(soup.select('#acrCustomerReviewText')[0].get_text().split(' ')[0].replace(".", ""))
                except:
                    review_score = ''
                    review_count = ''
            
            # checking if there is "Out of stock"
            try:
                soup.select('#availability .a-color-state')[0].get_text().strip()
                stock = 'Out of Stock'
            except:
                # checking if there is "Out of stock" on a second possible position
                try:
                    soup.select('#availability .a-color-price')[0].get_text().strip()
                    stock = 'Out of Stock'
                except:
                    # if there is any error in the previous try statements, it means the product is available
                    stock = 'Available'

            log = pd.DataFrame({'date': now.replace('h',':').replace('m',''),
                                'code': prod_tracker.code[x], # this code comes from the TRACKER_PRODUCTS file
                                'url': url,
                                'title': title,
                                'buy_below': prod_tracker.buy_below[x], # this price comes from the TRACKER_PRODUCTS file
                                'price': price,
                                'stock': stock,
                                'review_score': review_score,
                                'review_count': review_count}, index=[x])

            try:
                # This is where you can integrate an email alert!
                if price < prod_tracker.buy_below[x]:
                    print('************************ ALERT! Buy the '+prod_tracker.code[x]+' ************************')
            
            except:
                # sometimes we don't get any price, so there will be an error in the if condition above
                pass

            tracker_log = tracker_log.append(log)
            print('appended '+ prod_tracker.code[x] +'\n' + title + '\n\n')            
            sleep(5)
        
        interval += 1# counter update
        
        sleep(interval_hours*1*1)
        print('end of interval '+ str(interval))
    
    # after the run, checks last search history record, and appends this run results to it, saving a new file
    filepath = "C:/Users/Allston.Fojas/Desktop/amazon_webscraper-master/amazon_webscraper-master/search_history/*.xlsx" #'C:/Users/Fábio/Documents/Python Projects/202005 Amazon Webscraper/search_history/*.xlsx'
    last_search = glob(filepath)[-1] # path to file in the folder
    search_hist = pd.read_excel(last_search)
    final_df = search_hist.append(tracker_log, sort=False)
    
    final_df.to_excel('search_history/SEARCH_HISTORY_{}.xlsx'.format(now), index=False)
    print('end of search')

In [20]:
test_df = pd.read_excel("search_history/SEARCH_HISTORY_2020-07-29 19h13m.xlsx", encoding="utf-8", errors="ignore")

In [21]:
display(test_df.shape)
test_df.head()

(80, 10)

,date,code,url,title,price,alert,review_score,review_count,buy_below,stock
0,2020-07-29 19:13,snow_sled,https://www.amazon.com/ANYOYO-Outdoor-Toboggan...,ANYOYO Kids Outdoor Plastic Sport Toboggan Win...,NaN,NaN,4.2,14.0,10.0,Available
1,2020-07-29 19:13,airhead_classic_snow_sled,https://www.amazon.com/AIRHEAD-CLASSIC-Plastic...,Airhead Classic | 1-2 Person Snow Sled,NaN,NaN,4.5,429.0,10.0,Available
2,2020-07-29 19:13,slippery_racer_snow_sled,https://www.amazon.com/Slippery-Racer-Downhill...,Slippery Racer Downhill Xtreme Toboggan Snow Sled,NaN,NaN,4.6,368.0,30.0,Available
3,2020-07-29 19:13,flyer_611_snow_sled,https://www.amazon.com/Flexible-Flyer-Lightwei...,Flexible Flyer 611 Flying Carpet Lightweight R...,NaN,NaN,3.9,389.0,5.0,Available
4,2020-07-29 19:13,adudu_47_snow_tube,https://www.amazon.com/DUDU-Snow-Tube-Inflatab...,A-DUDU Snow Tube - Super Big 47 Inch Inflatabl...,27.99,NaN,NaN,NaN,20.0,Available
